# TUIA / AAII - TP2: Redes Recurrentes y Transformers - Problema 2b

> *Modelo de generación de textos basados en RNN para escribir noticias falsas.*


# Procedimientos Previos

## Preparación del entorno

### Librerías

In [ ]:
# Data
import numpy as np
import os
import re
import tensorflow as tf
import tensorflow_datasets as tfds

# Redes Neuronales
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

2024-07-04 11:29:11.229389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 11:29:11.229520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 11:29:11.367659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### GPU

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

## Dataset

El dataset proporcionado incluye 120000 artículos correspondientes a 4 categorías distintas. En este caso, la categoría no es relevante, sólo utilizaremos el dataset como un cuerpo de texto para entrenar un modelo recurrente de generación de texto.
https://www.tensorflow.org/datasets/catalog/ag_news_subset

In [ ]:
df, info = tfds.load('ag_news_subset', with_info=True,
                          as_supervised=True)
train_ds, val_ds = df['train'], df['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteULKK7J/ag_news_subset-train.tfrecord*...:  …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteULKK7J/ag_news_subset-test.tfrecord*...:   …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Función para extraer el texto del dataset
def extract_text(data):
    text = ''
    for article, _ in data:
        text += article.numpy().decode('utf-8') + ' '
    return text

# Extraer el texto de los conjuntos de entrenamiento y prueba
train_text = extract_text(train_ds)
test_text = extract_text(val_ds)

# Unir todos los textos en uno solo
text = train_text + test_text

# Mostrar una muestra del texto
print(text[:500])

AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions. Reuters - Major League Baseball\Monday announced a decision on the appeal filed by Chicago Cubs\pitcher Kerry Wood regarding a suspension stemming from an\incident earlier this season. President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local


### Limpieza

In [ ]:
def clean_text(text):
    # Conservar solo letras, números, espacios y ciertos caracteres de puntuación
    text = re.sub(r'[^a-zA-Z0-9.,!? ]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\[UNK\]', ' ', text)
    return text


# Aplicar limpieza al texto
cleaned_text = clean_text(text)

# Verificación de la limpieza
print("Texto original:", text[:500])  # Mostrar una muestra del texto original
print("Texto limpio:", cleaned_text[:500])  # Mostrar una muestra del texto limpio

Texto original: AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions. Reuters - Major League Baseball\Monday announced a decision on the appeal filed by Chicago Cubs\pitcher Kerry Wood regarding a suspension stemming from an\incident earlier this season. President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local
Texto limpio: AMD 39s new dualcore Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions. Reuters Major League BaseballMonday announced a decision on the appeal filed by Chicago Cubspitcher Kerry Wood regarding a suspension stemming from anincident earlier this season. President Bush 39s quotrevenueneutral quot tax reform needs losers to balance its winners, and people claiming the federal deduction fo

### Procesamiento

In [ ]:
# Función para agregar tokens de inicio y fin a cada oración
def add_start_end_tokens(sentences):
    return ['<start> ' + sentence + ' <end>' for sentence in sentences]

# Crear tokenizer y ajustarlo a las oraciones con tokens
def create_tokenizer(sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer

# Calcular la longitud máxima de las descripciones
def max_length(sentences):
    return max(len(sentence.split()) for sentence in sentences)

# Crear vocabulario a partir de las oraciones
def text_vocabulary(sentences):
    vocab = set()
    for sentence in sentences:
        vocab.update(sentence.split())
    return vocab

def reduce_sentences(sentences):
    percentage = 5  # Ajusta este porcentaje según sea necesario
    subset_size = int(len(sentences) * (percentage / 100))
    reduced_sentences = [sentence.strip() for sentence in sentences[:subset_size] if sentence.strip()]
    return reduced_sentences

In [ ]:
print("Longitud de cleaned_text:", len(cleaned_text))
# Dividir el texto limpiado en oraciones utilizando el punto como delimitador
sentences = cleaned_text.split('.')
print("Número de oraciones después de dividir por puntos:", len(sentences))

Longitud de cleaned_text: 24265313
Número de oraciones después de dividir por puntos: 219461


In [ ]:
# Reducir las oraciones y almacenar en una variable diferente
reduced_sentences_list = reduce_sentences(sentences)

# Agregar tokens de inicio y fin a cada oración reducida
reduced_sentences_with_tokens = add_start_end_tokens(reduced_sentences_list)

# Crear un tokenizer usando las oraciones con tokens
tokenizer = create_tokenizer(reduced_sentences_with_tokens)

# Crear vocabulario a partir de las oraciones
vocab = text_vocabulary(reduced_sentences_with_tokens)
vocab_size = len(vocab) + 1  # +1 para la palabra desconocida

# Calcular la longitud máxima de las oraciones
max_sentence_length = max_length(reduced_sentences_with_tokens)

print("Vocabulario size:", vocab_size)
print("Longitud máxima de oraciones:", max_sentence_length)

Vocabulario size: 26994
Longitud máxima de oraciones: 122


## Modelo

### Definición

In [ ]:
# Modelo de generación de texto
def define_model(vocab_size, max_length):
    inputs = Input(shape=(max_length,))
    x = Embedding(vocab_size, 256, mask_zero=True)(inputs)
    x = Dropout(0.3)(x)
    x = LSTM(256, return_sequences=True, use_cudnn=False)(x)  # Desactivar cuDNN aquí
    x = Dropout(0.3)(x)
    x = LSTM(256, use_cudnn=False)(x)  # Desactivar cuDNN aquí
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(vocab_size, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = define_model(vocab_size, max_sentence_length)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 122)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 122, 256)  │  6,910,464 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 122, 256)  │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 122)       │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 122, 256)  │    525,312 │ dropout[0][0],    │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 122, 256)  │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     65,792 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 26994)     │  6,937,458 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,964,338 (57.08 MB)

 Trainable params: 14,964,338 (57.08 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Función generadora de datos
def data_generator(tokenizer, max_length, sentences, batch_size):
    while True:
        for i in range(0, len(sentences), batch_size):
            batch_sentences = sentences[i:i+batch_size]
            X, y = create_sequences(tokenizer, max_length, batch_sentences)
            yield X, y

# Crear secuencias de entrada y salida para el entrenamiento
def create_sequences(tokenizer, max_length, sentences):
    X, y = [], []
    for sentence in sentences:
        encoded = tokenizer.texts_to_sequences([sentence])[0]
        for i in range(1, len(encoded)):
            X.append(pad_sequences([encoded[:i]], maxlen=max_length, padding='pre')[0])
            y.append(to_categorical([encoded[i]], num_classes=vocab_size)[0])
    return np.array(X), np.array(y)

### Entrenamiento

In [ ]:
checkpoint = ModelCheckpoint('model_best.keras', monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

In [ ]:
batch_size = 32
generator = data_generator(tokenizer, max_sentence_length, reduced_sentences_with_tokens, batch_size)
model.fit(generator, steps_per_epoch=len(reduced_sentences_with_tokens) // batch_size, epochs=300, callbacks=[checkpoint, early_stopping, reduce_lr])

# Guardar el modelo
model.save("WordLevelTextGenModel.h5")

# Generar texto
def generate_text(model, tokenizer, input_text, max_length, num_words=30):
    for _ in range(num_words):
        encoded = tokenizer.texts_to_sequences([input_text])[0]
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        prediction = np.argmax(model.predict(encoded), axis=-1)
        predicted_word = tokenizer.index_word[prediction[0]] if prediction[0] > 0 else ''
        if predicted_word == 'end':
            break
        input_text += ' ' + predicted_word
    return input_text

generated_text = generate_text(model, tokenizer, '<start> a man', max_sentence_length)
print("Generated text:", generated_text)

Epoch 1/300


I0000 00:00:1720092646.418492     120 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720092646.449497     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720092646.471928     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720092646.527311     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720092646.584665     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


  3/324 ━━━━━━━━━━━━━━━━━━━━ 34:00 6s/step - loss: 10.2027

W0000 00:00:1720092659.201100     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720092665.572141     123 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720092665.645386     123 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720092665.703992     123 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


  4/324 ━━━━━━━━━━━━━━━━━━━━ 33:59 6s/step - loss: 10.2023

...

  1/324 ━━━━━━━━━━━━━━━━━━━━ 37:11 7s/step - loss: 7.2427

W0000 00:00:1720094265.927657     123 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


324/324 ━━━━━━━━━━━━━━━━━━━━ 84s 238ms/step - loss: 7.0627 - learning_rate: 0.0010
Epoch 3/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 6.8207 - learning_rate: 0.0010
Epoch 4/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 6.6527 - learning_rate: 0.0010
Epoch 5/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 6.5143 - learning_rate: 0.0010
Epoch 6/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 6.3813 - learning_rate: 0.0010
Epoch 7/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 6.2564 - learning_rate: 0.0010
Epoch 8/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 6.1349 - learning_rate: 0.0010
Epoch 9/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 239ms/step - loss: 6.0266 - learning_rate: 0.0010
Epoch 10/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 5.9076 - learning_rate: 0.0010
Epoch 11/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - loss: 5.8019 - learning_rate: 0.0010
Epoch 12/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 238ms/step - l

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



324/324 ━━━━━━━━━━━━━━━━━━━━ 78s 240ms/step - loss: 4.0228 - learning_rate: 0.0010
Epoch 58/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 78s 239ms/step - loss: 3.9975 - learning_rate: 0.0010
Epoch 59/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 78s 240ms/step - loss: 3.9798 - learning_rate: 0.0010
Epoch 60/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 239ms/step - loss: 3.9556 - learning_rate: 0.0010
Epoch 61/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 239ms/step - loss: 3.9368 - learning_rate: 0.0010
Epoch 62/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 77s 239ms/step - loss: 3.9146 - learning_rate: 0.0010
Epoch 63/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 79s 243ms/step - loss: 2.8620 - learning_rate: 0.0010
Epoch 207/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 79s 243ms/step - loss: 2.8539 - learning_rate: 0.0010
Epoch 208/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 78s 242ms/step - loss: 2.8644 - learning_rate: 0.0010
Epoch 209/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 79s 242ms/step - loss: 2.8699 - learning_rate: 0.0010
Epoch 210/300
324/324 ━━━━━━━━━━━━━━━━━━━━ 78s 242

## Generación

In [ ]:
generated_text = generate_text(model, tokenizer, '<start> like', max_sentence_length)
print("Generated text:", generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Generated text: <start> like the lash inspired the

### Ejemplos de generaciones de texto



*   "OpenAI recently guitarworld magazine published a new videotaped general to be a new antimultiple sclerosis drug antegren developed by biogen"
*   "OpenAI CEO says it is now reaching the forthcoming break for head of the new york yankees"
*   "OpenAI buy the scoreline and anders limpars winner were forgotten when the allstar ninth round of the utah press on monday"
*   "Argentina last month 39s new york islanders has fired that the squirrel and a new version of the 21stranked leadership companies that could be a new companion music associations available multiple than any"
*   "Argentina just announced the dow jones industrial is mostly to be a new companion music firms available to plug tech emissions a new technology in the world'

